In [ ]:
import os
import sys

sys.path.append(os.path.abspath("src/"))

import logging
import sys

import hydra
import matplotlib.pyplot as plt
import mlflow
import numpy as np
import pandas as pd
import torch
import yaml
from hydra import initialize_config_dir
from joblib import Memory
from omegaconf import DictConfig, OmegaConf

from evaluators import Evaluator
from models import FastTextWrapper
from src.datasets import SoftClassifDataset, TextClassificationDataModule
from utils.data import PATHS, get_df_naf, get_file_system, get_processed_data, get_test_data, get_Y
from utils.evaluation import (
    get_fasttext_preds,
    get_ground_truth,
    get_label_mapping,
    sort_and_get_pred,
)
from utils.mappings import mappings
from utils.mlflow import create_or_restore_experiment
from utils.validation_viz import (
    calibration_curve,
    confidence_histogram,
    get_automatic_accuracy,
)

%load_ext autoreload
%autoreload 2

In [ ]:
os.environ['MLFLOW_TRACKING_URI'] = "https://projet-ape-mlflow.user.lab.sspcloud.fr/" 
# model_name = "FastText-pytorch"
# module = mlflow.pytorch.load_model(f"models:/{model_name}/latest")

run_id = "01fd012d1d8f45828a889efd8cb926ec"
logged_model = f'runs:/{run_id}/model'
# Load model as a PyFuncModel.
module = mlflow.pytorch.load_model(logged_model)

In [ ]:
# Download the artifact directory (e.g., to a temp dir)
local_artifacts_path = mlflow.artifacts.download_artifacts(run_id=run_id)

# Load the YAML config
with open(f"{local_artifacts_path}/hydra_config.yaml", "r") as f:
    config = yaml.safe_load(f)

In [ ]:
cfg: DictConfig = OmegaConf.create(config)

In [ ]:
trainer = hydra.utils.instantiate(cfg.model.trainer)

In [ ]:
datamodule = TextClassificationDataModule(
                    cfg.data,
                    cfg.tokenizer,
                    cfg.dataset,
                    batch_size = 256)

In [ ]:
datamodule.setup()

In [ ]:
val_loader = datamodule.val_dataloader()
batch = next(iter(val_loader))

In [ ]:
predictions = trainer.predict(module, val_loader)
predictions_tensor = torch.cat(predictions)

In [ ]:
n_samples = 10000
true_values = torch.tensor(datamodule.df_val.apet_finale[:n_samples].values).cuda()

def high_proba_score(x, y):
    scores = 1 - x[list(range(len(x))), y]
    return scores

manually_coding_mask = predictions_tensor[:n_samples].max(-1).values <= 0.65

print("Taux de codification automatique : ", 1- (manually_coding_mask).float().mean().item())
print('Nombre de samples utilisés pour calibration : ', len(predictions_tensor[:n_samples][manually_coding_mask]))
scores = high_proba_score(predictions_tensor[:n_samples][manually_coding_mask].cuda(), true_values[manually_coding_mask])

In [ ]:
n = len(scores)
alpha = 0.05
q_level = np.ceil((n+1)*(1-alpha))/n
print(q_level)
threshold_cp = torch.quantile(scores.float(), q_level, interpolation="higher")
print(threshold_cp)

In [ ]:
test_loader = datamodule.test_dataloader()
batch = next(iter(test_loader))

In [ ]:
(1 - module.predict_step(batch, 0).cpu() <= threshold_cp.cpu()).sum(dim = -1).float().mean()
# (module.predict_step(batch, 0).cuda().sort(dim = -1, descending=True).values.cumsum(-1) <= threshold_cp).sum(dim = -1).float().mean()